In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

pd.set_option("display.max_columns",None)
df=sns.load_dataset("titanic")
df.head(5)

# Kategorik Verilerin Analizi

In [ ]:
# Dataframe'in genel özelliklerine göz atma

def check_df(dataframe,head=5):
    print("######## Shape #########")  # Dataframe'in satır ve sütun bilgisini ekrana yazdırma 
    print(dataframe.shape)
    print("######## Type #########")   # Dataframe de bulunan sütunların tiplerini ekrana yazdırma
    print(dataframe.dtypes)
    print("######## Head #########")   # Dataframe'in ilk 5 satırını ekrana yazdırma
    print(dataframe.head(head))
    print("######## Tail #########")   # Dataframe'in son 5 satırını ekrana yazdırma
    print(dataframe.tail(head))
    print("######## NaN #########")    # Dataframe de bulunan eksik verileri tespit etme
    print(dataframe.isnull().sum())
    print("######## Quantiles #########")   # Dataframe den yüzdelik dilimler halinde bilgi alma
    print(dataframe.describe([0,0.25,0.50,0.75,0.90,1]).T)
    
check_df(df)
    

In [ ]:
# Tip bilgisine göre kategorik olan sütunlara ulaşalım

cat_colm=[col for col in df.columns if str(df[col].dtypes) in ["category","object","bool"]]
print(cat_colm)

In [ ]:
# Numerik gibi görünen ama kategorik olan verileri seçelim (Örneğin bu veri setinde bulunan survived değişkeni 1 ve 0 larla ifade ediliyor fakat kategorik bir değişken olarak almalıyız!)

num_but_cat=[col for col in df.columns if df[col].nunique() < 10 and df[col].dtypes in ["int","float"]]
print(f"num_but_car olarak seçilen sütunlar:{num_but_cat}")

# Yukarıda yaptığımız işlemin tersini yaparak çok fazla değişkene sahip kategorik verileri çıkartıyoruz. (Burada sınırı 20 olarak seçtik sizler değiştirebilirsiniz.)

cat_but_car=[col for col in df.columns if df[col].nunique() > 20 and df[col].dtypes in ["category","object"]]
print(f"cat_but_car olarak seçilen sütunlar: {cat_but_car}")

# Verileri birleştiriyoruz
cat_colm = [col for col in cat_colm if col not in cat_but_car]


print(cat_colm)

In [ ]:
# Grafiklerle kategorik sütunlarımızın yüzdelik dağılımlarını gösterelim

def cat_summary(dataframe,col_name,plot=False):
    print(pd.DataFrame({col_name:dataframe[col_name].value_counts(),
                      "%'lik Oran":100*df[col_name].value_counts() / len(df)}))
    print("################################################")
    if plot:
        sns.countplot(x=dataframe[col_name],data=dataframe)
        plt.show(block=True)
        
        
# Boolean türde olan veriler sorun çıkartmaması için tip dönüşümü yaptık
for col in cat_colm:
    if df[col].dtypes=="bool":
        df[col]=df[col].astype(int)
        cat_summary(df,col,plot=True)
    else:
        cat_summary(df,col,plot=True)

# Sayısal Verilerin Analizi

In [ ]:
# Sayısal verilerin bulunduğu sütunları seçiyoruz

num_colm=[col for col in df.columns if df[col].dtypes in ["int","float"]]
print(num_cols)

# Kategorik olup numerik gibi görünen sütunları siliyoruz
num_colm = [col for col in num_colm if col not in num_but_cat]
print(num_colm)

In [ ]:
# Sayısal verilerimizi grafiksel olarak analiz edelim
def num_summary(dataframe,numerical_col,plot=False):
    print(df[numerical_col].describe().T)
    
    if plot:
        dataframe[numerical_col].hist()
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show(block=True)
    print("#####################################")
    
for col in num_colm:
    num_summary(df,col,plot=True)

# Hedef Değişken Analizi

In [ ]:
# Kategorik değişkenlerle hedef değişken analizi

def target_summary_with_cat(dataframe,target,categorical_col):
    print(pd.DataFrame({"TARGET_MEAN":dataframe.groupby(categorical_col,observed=True)[target].mean()}))
    
    
for col in cat_colm:
    target_summary_with_cat(df,"survived",col)

In [ ]:
# Sayısal değişkenlerle hedef değişken analizi

def target_summary_with_num(dataframe,target,numerical_col):
    print(dataframe.groupby(target,observed=True).agg({numerical_col:"mean"}),end="\n\n\n")
    
    
for col in num_colm:
    target_summary_with_num(df,"survived",col)

# Korelasyon Analizi

In [ ]:
corr=df[num_colm].corr()

# Isı haritası

# Bu veri setinde sadece 2 adet sayısal veri geldiği için ısı haritası küçük fakat farklı veri setlerinde deneyebilirsiniz :)
sns.set(rc={"figure.figsize":[12,12]})

sns.heatmap(corr,cmap="RdBu",annot=True,fmt='.2f')
plt.show()